<h1>MBS Quotation Data Proceccing and Sorting<h1\>
    

This project aims to assist fund manager to manage Mortgage backed Securities (MBS) quotation data. The MBS quotes came from different agncies (e.g. Citi, JP Morgan, Wells Fargo...), and each agency provide a list of MBS quotation in different formats. The manager want to find out how many agencies offering the same security under certain criteria (Pools,Term,Coupon rates).  

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

import portfolioopt as pfopt

import time
import pandas as pd
import numpy as np
from numpy import split
from numpy import array
from numpy import log
from numpy import zeros
from numpy import concatenate
from scipy import linalg

from matplotlib import pyplot as plt
from datetime import datetime
from math import sqrt

from pandas.tseries.offsets import BMonthEnd, BQuarterEnd, BMonthBegin, BQuarterBegin, BQuarterEnd, BDay
from pandas import read_csv, DataFrame, concat, datetime
from pandas.tseries.offsets import BMonthEnd, BQuarterEnd, MonthBegin, BQuarterBegin, BQuarterEnd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [192]:
# import os
sCurrentWorkingDirectory = os.getcwd()
print('CurrentWorkingDirectory: ', sCurrentWorkingDirectory, '\n')
sDir = sCurrentWorkingDirectory

# load data
def parse(x):
    return datetime.strptime(x, '%Y/%m/%d')

# PW_data = read_csv('t50_price_weighting_rawdata.csv',  parse_dates = ['date'], date_parser=parse)
# earning_data = read_csv('earning_rawdata.csv',  parse_dates = ['date'], date_parser=parse)

CurrentWorkingDirectory:  D:\Jupyter\MBS_Data_Processing 



In [193]:
import datetime

# now = datetime.datetime.now()
# date = datetime.datetime.now().strftime("%Y-%m-%d")
date = '2019-03-25'

In [194]:
# Wells Fargo Securities
xls = pd.ExcelFile('WFSPassThroughOfferingsAsia-2019-3-25.xls')

# WFS's file has two sheets
df = pd.read_excel(xls, sheet_name = 'All Spec Pools', header= 13)
df = df.iloc[:,:-3]
df = df.drop('Item', axis=1)
df.insert(0,'Date', date)
df.insert(1,'Offer', 'WFS')
df.insert(2,'Agcy',df['Pool'].str.slice(stop=2))

# display(df.head())
# display(df.tail())

# extract the columns we need
WFS = df[['Date','Offer','Pool','Agcy','Term','Coupon','Issue Date']].reset_index().drop('index',axis=1)
display(WFS.head())
display(WFS.tail())

,Date,Offer,Pool,Agcy,Term,Coupon,Issue Date
0,2019-03-25,WFS,FN BE3908,FN,30,2.5,2016-12-01
1,2019-03-25,WFS,FN AS8270,FN,30,2.5,2016-10-01
2,2019-03-25,WFS,FN BD4759,FN,30,2.5,2016-08-01
3,2019-03-25,WFS,FN BD5508,FN,30,2.5,2016-08-01
4,2019-03-25,WFS,FN MA2764,FN,30,2.5,2016-08-01


,Date,Offer,Pool,Agcy,Term,Coupon,Issue Date
273,2019-03-25,WFS,FN MA3591,FN,20,4.5,2019-01-01
274,2019-03-25,WFS,FN BM1022,FN,10,2.5,2017-03-01
275,2019-03-25,WFS,FN MA1139,FN,10,2.5,2012-07-01
276,2019-03-25,WFS,FN MA3221,FN,10,3.0,2017-11-01
277,2019-03-25,WFS,FN CA0373,FN,30,3.0,2017-08-01


In [195]:
# add Issue Date for Citi files
def assign_month(m):
    if m == 'Q1':
        d ='-01-01'
    elif m=='Q2':
        d ='-04-01'
    elif m=='Q3':
        d ='-07-01'
    else:
        d ='-10-01'
    return d

In [202]:
# Citi has two files

# processing file 1:
# 20190325_Specified; included 3 sheets; required to add Issue Date,it only has Orig, i.e.'18 Q3
xls = pd.ExcelFile('20190325_Specified.xls')

sheets = ['CONV','15yr','GNMA']
sheets_data = []
for i in range(len(sheets)):
    df = pd.read_excel(xls, sheet_name = sheets[i], header= 5)

    df.insert(0,'Date', date)
    df.insert(1,'Offer', 'Citi')
    df.insert(2,'Agcy',df['Pool'].str.slice(stop=2))
    df.insert(3,'Issue Date',pd.to_datetime(('20' + df['Orig'].str.slice(start=1,stop=3) + df['Orig'].str.slice(start=-1).apply(assign_month))))
    df = df[['Date','Offer','Pool','Agcy','Term','Cpn','Issue Date']]
    df.columns = ['Date','Offer','Pool','Agcy','Term','Coupon','Issue Date']
    sheets_data.append(df)

# processing file 2:
# Large_Specified_GNMA_20190326.xlsm; it has only one sheet 'GN'
# delete the last five rows
xls = pd.ExcelFile('Large_Specified_GNMA_20190326.xlsm')
df = pd.read_excel(xls, sheet_name = 'GN', header= 5)
df = df.iloc[:-5,]

df.insert(0,'Date', date)
df.insert(1,'Offer', 'Citi')
df.insert(2,'Agcy',df['Pool'].str.slice(stop=2))
df.insert(3,'Issue Date',pd.to_datetime(('20' + df['Orig'].str.slice(start=1,stop=3) + df['Orig'].str.slice(start=-1).apply(assign_month))))
df = df[['Date','Offer','Pool','Agcy','Term','Cpn','Issue Date']]
df.columns = ['Date','Offer','Pool','Agcy','Term','Coupon','Issue Date']
sheets_data.append(df)

# integrating 
Citi = pd.concat(sheets_data).reset_index().drop('index',axis=1)
display(Citi.head())
display(Citi.tail())

# export Citi
# Citi.to_csv('citi.csv')

,Date,Offer,Pool,Agcy,Term,Coupon,Issue Date
0,2019-03-25,Citi,FN AB7517,FN,20.0,2.5,2012-10-01
1,2019-03-25,Citi,FN AB7673,FN,20.0,2.5,2012-10-01
2,2019-03-25,Citi,FN AS8773,FN,20.0,3.0,2017-10-01
3,2019-03-25,Citi,FG C91955,FG,20.0,3.0,2017-10-01
4,2019-03-25,Citi,FG C91949,FG,20.0,3.0,2017-10-01


,Date,Offer,Pool,Agcy,Term,Coupon,Issue Date
328,2019-03-25,Citi,G2 MA5712,G2,30.0,5.0,2018-10-01
329,2019-03-25,Citi,G2 MA5765,G2,30.0,5.0,2018-10-01
330,2019-03-25,Citi,G2 MA5819,G2,30.0,5.0,2019-10-01
331,2019-03-25,Citi,G2 MA5878,G2,30.0,5.0,2019-10-01
332,2019-03-25,Citi,G2 MA5713,G2,30.0,5.5,2018-10-01


In [203]:
# J.P. Morgan
xls = pd.ExcelFile('jpm pass-thru offerings 03-25-19.xls')
df = pd.read_excel(xls, sheet_name = 'mbs', header= 6)
df = df.iloc[:,:24]
df = df.reset_index()
df = df.drop('level_1', axis=1)
df = df.dropna(subset=['level_0'])

df.insert(0,'Date', date)
df.insert(1,'Offer', 'JPM')
df.insert(2,'Pool', df['Agcy'] + ' ' + df['Pool #'])
df.insert(3,'Issue Date',np.nan)
JPM = df[['Date','Offer','Pool','Agcy','Term','Cpn','Issue Date']]
JPM.columns = ['Date','Offer','Pool','Agcy','Term','Coupon','Issue Date']
JPM = JPM.reset_index().drop('index',axis=1)

display(JPM.head())
display(JPM.tail())

# 匯出JPM
# JPM.to_csv('JPM.csv')

,Date,Offer,Pool,Agcy,Term,Coupon,Issue Date
0,2019-03-25,JPM,FN BM5658,FN,30,4.5,NaN
1,2019-03-25,JPM,FG G61825,FG,30,4.5,NaN
2,2019-03-25,JPM,FG V85163,FG,30,4.5,NaN
3,2019-03-25,JPM,FN BN0340,FN,30,4.5,NaN
4,2019-03-25,JPM,FG V85080,FG,30,4.5,NaN


,Date,Offer,Pool,Agcy,Term,Coupon,Issue Date
406,2019-03-25,JPM,G2 BF1208,G2,30,5.0,NaN
407,2019-03-25,JPM,G2 BI4219,G2,30,5.0,NaN
408,2019-03-25,JPM,G2 BK0853,G2,15,3.0,NaN
409,2019-03-25,JPM,G2 MA5736,G2,15,3.0,NaN
410,2019-03-25,JPM,G2 MA5551,G2,15,4.0,NaN


<h2>Result

In [209]:
pd.concat([WFS,Citi,JPM]).to_csv('agg_dataset.csv')

In [211]:
df = pd.concat([WFS,Citi,JPM])
df = df.set_index(['Date','Term','Coupon','Agcy','Pool']).sort_index()
# df.to_csv('aggregated_dataset.csv')
# display(df)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

Offer Issue Date
Date       Term Coupon Agcy Pool                      
2019-03-25 10.0 2.5    FN   FN BM1022   WFS 2017-03-01
                            FN BM3056   JPM        NaT
                            FN MA1139   WFS 2012-07-01
                            FN MA3079   JPM        NaT
                3.0    FG   FG J39953   JPM        NaT
                       FN   FN MA3221   WFS 2017-11-01
                            FN MA3531   JPM        NaT
                            FN MA3653   JPM        NaT
                3.5    FG   FG G16768  Citi 2018-10-01
                       FN   FN MA3623   JPM        NaT
                            FN MA3650   JPM        NaT
                4.0    FN   FN BM5515   JPM        NaT
           15.0 2.0    FG   FG G14660   JPM        NaT
                            FG G16580   WFS 2018-08-01
                            FG J22586   WFS 2013-02-01
                            FG J25759   WFS 2013-09-01
                       FN   FN AS8787   JPM        NaT
                            FN BE0974   WFS 2016-11-01
                            FN BE5623   WFS 2017-01-01
                            FN BE5627   JPM        NaT
                            FN MA2774   JPM        NaT
                            FN MA2867   JPM        NaT
                2.5    FG   FG G16084  Citi 2016-10-01
                            FG G16350   WFS 2017-10-01
                            FG G16566   WFS 2018-07-01
                            FG G16595  Citi 2017-10-01
                            FG G16597   JPM        NaT
                            FG G16659   WFS 2018-11-01
                            FG G16660   WFS 2018-11-01
                            FG G16661   WFS 2018-11-01
                            FG G18654   WFS 2017-07-01
                            FG G18665   WFS 2017-10-01
                            FG G18676   WFS 2018-01-01
                            FG J29624   JPM        NaT
                            FG J34688   WFS 2016-06-01
                            FG J35334   JPM        NaT
                            FG J37306   JPM        NaT
                            FG J38059   WFS 2017-11-01
                            FG J38106   JPM        NaT
                            FG U41473   JPM        NaT
                            FG U70679   JPM        NaT
                            FG V61548   JPM        NaT
                       FN   FN AS4480   WFS 2015-02-01
                            FN AS4486   WFS 2015-02-01
                            FN AS4713  Citi 2015-10-01
                            FN AS4751  Citi 2015-10-01
                            FN AS4939   WFS 2015-04-01
                            FN AS4941  Citi 2015-10-01
                            FN AS5148   WFS 2015-05-01
                            FN AS5297   JPM        NaT
                            FN AS5298   WFS 2015-06-01
                            FN AS5783   WFS 2015-08-01
                            FN AS5896   JPM        NaT
                            FN AS6054   JPM        NaT
                            FN AS6056   JPM        NaT
                            FN AS6713   JPM        NaT
                            FN AS6715   JPM        NaT
                            FN AS6873   JPM        NaT
                            FN AS6951   WFS 2016-03-01
                            FN AS7225   JPM        NaT
                            FN AS7495   WFS 2016-06-01
                            FN AS7830   WFS 2016-08-01
                            FN AS8011   WFS 2016-09-01
                            FN AS8012   WFS 2016-09-01
                            FN AS8046   WFS 2016-09-01
                            FN AS8194   WFS 2016-10-01
                            FN AS8208   WFS 2016-10-01
                            FN AS8210   WFS 2016-10-01
                            FN AS8240   WFS 2016-10-01
                            FN AS8242   WFS 2016-10-01
                            FN AS8245   WFS 2016-10-01
                       